# 生成 AI による記事アイデアの作成

この Notebook では、生成 AI に記事のネタだしから調査、記事作成までを自動で実行する Agent を作成します。

PV 数の多い記事を分析して得た PV 数が高くなりそうな記事のネタの仮説を元に、Web 検索を利用し情報を集めて記事アイデアを提案するという設計にします。

## Agent の作成

まずは、 [こちら](https://ap-northeast-1.console.aws.amazon.com/bedrock/home?region=ap-northeast-1#/agents)から Amazon Bedrock のコンソール画面に移動し、エージェントを作成します。
`エージェントの作成` をクリックし、名前を「search-agent」にして `作成` をクリックします。

![](images/agent-1.png)


作成後に表示されるエージェントビルダーから、Agent の詳細設定を行います。
- `モデルを選択` では Claude 3.5 Sonnet を選択します。（モデルアクセスの有効化が済んでいない場合は[こちら](https://ap-northeast-1.console.aws.amazon.com/bedrock/home?region=ap-northeast-1#/modelaccess)から有効化してください）
- `エージェント向けの指示` では、以下の指示をコピー & ペーストしてください。
```
あなたは指示に応えるアシスタントです。 指示に応えるために必要な情報が十分な場合はすぐに回答し、不十分な場合は検索を行い必要な情報を入手し回答してください。複数回検索することが可能です。
```
![](images/agent-2.png)
- `アクショングループ` から `追加` をクリックし、`アクショングループを作成` 画面に移動します。
- アクショングループ名に `search-web` と入力します。説明には「与えられたクエリーで Web 検索を行い、結果を返します。」と入力します。
- `アクショングループの呼び出し` は `新しい Lambda 関数をすばやく作成する - 推奨` を選択します
- `アクショングループ関数 1` の名前に `search-web` と入力します。説明には以下の指示をコピー & ペーストしてください。
```
キーワードで検索し情報を取得します。調査、調べる、Xについて教える、まとめるといったタスクで利用できます。会話から検索キーワードを推測してください。検索結果には関連度の低い内容も含まれているため関連度の高い内容のみを参考に回答してください。複数回実行可能です。
```
- `パラメータ` に `query` という名前のパラメータを追加します。説明に「Web検索用のクエリー」と入力します。タイプは Srting とします。
- 画面下部にある `作成` をクリックします。
- 作成後、画面上部にある `保存` をクリックします。
![](images/agent-3.png)

## Lambda 関数の作成
- `設定 > 環境変数` から `TAVILY_API_KEY` というキーの環境変数を作成し、ハンズオンオペレーターから共有された Tavily の API キーを `値` に入力してください。
- `コードソース` で以下のコードを上書きし、`Deploy` をクリックします。

```python
import json
import urllib.request
import os

def lambda_handler(event, context):
    actionGroup = event['actionGroup']
    function = event['function']

    # イベントパラメーターから検索クエリを取得
    query = next(
        (item["value"] for item in event["parameters"] if item["name"] == "query"), ""
    )

    # Tavily APIを使用して検索を実行
    try:
        results = tavily_search(query)
        
        responseBody = {
            "TEXT": {
                "body": f"検索結果: {json.dumps(results, ensure_ascii=False)}"
            }
        }
    except Exception as e:
        responseBody = {
            "TEXT": {
                "body": f"検索中にエラーが発生しました: {str(e)}"
            }
        }

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print(f"Response: {json.dumps(function_response, ensure_ascii=False)}")

    return function_response

def tavily_search(query: str) -> dict:
    try:
        url = 'https://api.tavily.com/search'
        headers = {
            'Content-Type': 'application/json'
        }
        data = json.dumps({
            'api_key': os.environ.get('TAVILY_API_KEY'),
            'query': query,
            'search_depth': 'advanced',
            'include_answer': False,
            'include_images': False,
            'include_raw_content': False,
            'max_results': 3,
            'include_domains': [],
            'exclude_domains': []
        }).encode('utf-8')

        req = urllib.request.Request(url, data=data, headers=headers, method='POST')
        with urllib.request.urlopen(req) as response:
            body = response.read()
        return json.loads(body)
    except Exception as e:
        raise Exception(f"検索中にエラーが発生しました: {str(e)}")
```

## Agent のテスト
- `search-agent` のページに移動し、下部の `エイリアス` からエイリアスの作成を行います。エイリアス名は「v1」とします。
- 作成後に表示されるエイリアス ID と、`エージェントの概要` に表示されている `ID` をメモします。これらを用いて Agent の動作確認を行います。
![](images/agent-4.png)

In [ ]:
import json
import boto3
from uuid import uuid4

client = boto3.client('bedrock-agent-runtime', region_name='ap-northeast-1')

def invoke_agent(prompt: str):    
    agent_id = 'YOUR_AGENT_ID'
    agent_alias_id = 'YOUR_AGENT_ALIAS_ID'
    session_id = str(uuid4())
    
    response = client.invoke_agent(
        agentId=agent_id,
        agentAliasId=agent_alias_id,
        sessionId=session_id,
        inputText=prompt,
    )
    print("Trace:")
    for event in response['completion']:
        if 'trace' in event:
            data = event['chunk']['bytes']
            agent_answer = data.decode('utf8')
            return agent_answer
        elif 'chunk' in event:
            print(json.dumps(event['trace']))

In [ ]:
prompt = "今話題の日本の芸能人で話題になりそうなものを一つピックアップし、500文字程度のニュース記事を作成してください。"

answer = invoke_agent(prompt)
print(answer)

権限のエラーが出る場合は、[こちら](https://us-east-1.console.aws.amazon.com/iam/home?region=ap-northeast-1#/roles)より `AmazonSageMaker-ExecutionRole-2025...` というロールを検索し、それに対して `AmazonBedrockFullAccess` というポリシーを付与することによって実行可能になります。

## 記事アイデアの作成

Agent がインターネットから必要な情報を検索して記事を書く検証ができたため、次は PV 数が高い記事を分析した結果をもとに、ヒットする記事の仮説を立てて必要な情報を収集し記事の草案を作成させます。

In [ ]:
import pandas as pd
import os

output_path = "analysis.csv"
if os.path.exists(output_path):
    hits = pd.read_csv(output_path)
    for idx, row in hits.iterrows():
        print(row["analysis"])
        print("")

In [ ]:
for idx, row in hits[:3].iterrows():
    print("")
    print("##########################")
    print("--- PV 数の多い記事の要素 ---")
    print(row["analysis"])
    prompt = f"""あなたは記者です。PV 数の多い記事の要素をもとに記事のアイデアを作成し、必要な情報を調査し記事の原稿としてまとめあげて報告してください。
<PV 数の多い記事の要素>
{row["analysis"]}
</PV 数の多い記事の要素>"""
    answer = invoke_agent(prompt)
    print(answer)